In [1]:
import pandas as pd
import numpy as np
import csv

### Data gathered from:

- California WARN data: https://edd.ca.gov/Jobs_and_Training/Layoff_Services_WARN.htm
- population data from: https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-total.html#par_textimage_242301767

### This notebook will explore layoffs in counties with high rent burdens

In the [finding_ambiguous_cities](https://github.com/biglocalnews/WARN_Cali_analysis/blob/master/hardest_hit/finding_ambiguous_cities.ipynb) notebook, we've already standardized the county information. We've also identified a few records with the incorrect city name, so we need to clean those records. 

In [2]:
cali_layoffs = pd.read_csv('california_warn_raw_2.csv')
cali_population = pd.read_csv('county_population.csv', delimiter='\t', thousands=',')

In [3]:
cali_layoffs.head()

,Unnamed: 0,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,County Orig
0,0,06/09/2020,06/07/2020,07/01/2020,Bay Club Redondo Beach,Redondo Beach,Los Angeles County,102.0,Layoff Permanent,Los Angeles County
1,1,06/09/2020,06/07/2020,07/01/2020,Bay Club Rolling Hills,Rolling Hills Estates,Los Angeles County,64.0,Layoff Permanent,Los Angeles County
2,2,06/09/2020,06/07/2020,07/01/2020,Bay Club Santa Monica,Santa Monica,Los Angeles County,82.0,Layoff Permanent,Los Angeles County
3,3,06/19/2020,08/21/2020,07/01/2020,"Weber Metals, Inc",Paramount,Los Angeles County,169.0,Layoff Permanent,Los Angeles County
4,4,06/09/2020,06/07/2020,07/01/2020,StoneTree Golf Club,Novato,Marin County,32.0,Layoff Permanent,Marin County


In [4]:
cali_layoffs.tail()

,Unnamed: 0,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,County Orig
9671,9671,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",San Francisco,No Name,86.0,Layoff Permanent,No County Name
9672,9672,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",Santa Monica,No Name,49.0,Closure Permanent,No County Name
9673,9673,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",Venice,No Name,11.0,Closure Permanent,No County Name
9674,9674,06/29/2015,08/28/2015,06/30/2015,"Safeway, Inc.",Pleasanton,No Name,18.0,Layoff Unknown at this time,No County Name
9675,9675,06/30/2015,07/22/2015,06/30/2015,Medtronic Ablation Frontiers LLC,Carlsbad,No Name,50.0,Closure Permanent,No County Name


### California Layoffs

For the purposes of this analysis, we are selecting 2020 records by isolating the year on the `Notice date` column.
By choosing `Notice date` we make sure that these layoffs were issued after the start of shelter in place for 2020, as some layoff notices can be issues much earlier the layoff date.

In [5]:
cali_layoffs['Year'] = cali_layoffs['Notice Date'].str[-4:]
cali_layoffs.head()

,Unnamed: 0,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,County Orig,Year
0,0,06/09/2020,06/07/2020,07/01/2020,Bay Club Redondo Beach,Redondo Beach,Los Angeles County,102.0,Layoff Permanent,Los Angeles County,2020
1,1,06/09/2020,06/07/2020,07/01/2020,Bay Club Rolling Hills,Rolling Hills Estates,Los Angeles County,64.0,Layoff Permanent,Los Angeles County,2020
2,2,06/09/2020,06/07/2020,07/01/2020,Bay Club Santa Monica,Santa Monica,Los Angeles County,82.0,Layoff Permanent,Los Angeles County,2020
3,3,06/19/2020,08/21/2020,07/01/2020,"Weber Metals, Inc",Paramount,Los Angeles County,169.0,Layoff Permanent,Los Angeles County,2020
4,4,06/09/2020,06/07/2020,07/01/2020,StoneTree Golf Club,Novato,Marin County,32.0,Layoff Permanent,Marin County,2020


### Cleaning Layoff Types

Type Unknown and unknown at this time will be combined.

In [6]:
cali_layoffs['Layoff/Closure'].unique()

array(['Layoff Permanent', 'Layoff Type Unknown ', 'Layoff Temporary',
       'Closure Temporary', 'Closure Permanent', 'Closure Type Unknown ',
       'Closure Unknown at thistime', 'Layoff Unknown at thistime',
       'Layoff Unknown at this time', 'Closure Unknown at this time'],
      dtype=object)

In [7]:
cali_layoffs['Layoff/Closure clean'] = cali_layoffs['Layoff/Closure'].str.replace('at thistime', 'at this time')
cali_layoffs['Layoff/Closure clean'] = cali_layoffs['Layoff/Closure clean'].str.strip()
cali_layoffs['Layoff/Closure clean'] = cali_layoffs['Layoff/Closure clean'].str.lower()
cali_layoffs['Layoff/Closure clean'] = cali_layoffs['Layoff/Closure clean'].str.replace('unknown at this time', 'type unknown')
cali_layoffs['Layoff/Closure clean'].unique()

array(['layoff permanent', 'layoff type unknown', 'layoff temporary',
       'closure temporary', 'closure permanent', 'closure type unknown'],
      dtype=object)

In [8]:
years = ['2020', '2019']

### Extracting Only Necessary Records

Now that the data has been standardized, we will be selecting the records that have a notice date from the year 2020 or 2019. This will allow for comparisons between a pandemic and non-pandemic year in terms of layoff notices. 

In [9]:
cali_2020_2019 = cali_layoffs[cali_layoffs["Year"].isin(years)]
cali_2020_2019.head()

,Unnamed: 0,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,County Orig,Year,Layoff/Closure clean
0,0,06/09/2020,06/07/2020,07/01/2020,Bay Club Redondo Beach,Redondo Beach,Los Angeles County,102.0,Layoff Permanent,Los Angeles County,2020,layoff permanent
1,1,06/09/2020,06/07/2020,07/01/2020,Bay Club Rolling Hills,Rolling Hills Estates,Los Angeles County,64.0,Layoff Permanent,Los Angeles County,2020,layoff permanent
2,2,06/09/2020,06/07/2020,07/01/2020,Bay Club Santa Monica,Santa Monica,Los Angeles County,82.0,Layoff Permanent,Los Angeles County,2020,layoff permanent
3,3,06/19/2020,08/21/2020,07/01/2020,"Weber Metals, Inc",Paramount,Los Angeles County,169.0,Layoff Permanent,Los Angeles County,2020,layoff permanent
4,4,06/09/2020,06/07/2020,07/01/2020,StoneTree Golf Club,Novato,Marin County,32.0,Layoff Permanent,Marin County,2020,layoff permanent


In [10]:
# Checking for incorrect County in 'County' Column
# pd.options.display.max_rows = 999
# cali_2020_2019[cali_2020_2019['City'] == 'Sacramento']

### More Data Cleaning

A few records have the incorrect county, based on the information in the city column. These cities include: Sacramento, Hayward, Indian Wells, Cerritos, Vista, Roseville, City of Industry, Del Mar, Walnut Creek, and Los Gatos. For information on how these cities were selected, see the [finding_ambiguous_cities](https://github.com/biglocalnews/WARN_Cali_analysis/blob/master/hardest_hit/finding_ambiguous_cities.ipynb) notebook.

In [11]:
dict_cities = {
    'Sacramento':'Sacramento County',
    'San Francisco':'San Francisco County',
    'Los Angeles': 'Los Angeles County',
    'Hayward':'Alameda County',
    'Indian Wells':'Riverside County',
    'Cerritos':'Los Angeles County',
    'Vista':'San Diego County',
    'Roseville':'Placer County',
    'Berkeley': 'Alameda County',
    'City of Industry':'Los Angeles County',
    'Del Mar':'San Diego County',
    'Walnut Creek':'Contra Costa County',
    'Los Gatos':'Santa Clara County'
}

### Clean County Names
Below is our code to correct County Names based on City Names. 
We are taking the dictionary above, looping through the city and county names, which are the keys and values, replacing the incorrect County Name with the correct one for each city's record.

In [12]:
for city, county in dict_cities.items():
    cali_2020_2019.loc[cali_2020_2019['City'] == city, ['County']] = county

/Users/dilcia_mercedes/.local/share/virtualenvs/cali_WARN-MY_IDcvv/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [13]:
# Test to make sure Counties are named correctly. All records in 'County' column should now say sacramento - the 'County Orig' column should have the original County stated
sacramento_clean = cali_2020_2019[cali_2020_2019['City'] == 'Sacramento']
sacramento_clean.loc[1962:1963]

,Unnamed: 0,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,County Orig,Year,Layoff/Closure clean
1962,1962,03/17/2020,03/19/2020,06/05/2020,"Chico Community Publishing, Inc.",Sacramento,Sacramento County,45.0,Layoff Permanent,Del Norte County,2020,layoff permanent


In [14]:
cali_2020_2019.drop(columns='Unnamed: 0', inplace=True)
cali_2020_2019.head()

/Users/dilcia_mercedes/.local/share/virtualenvs/cali_WARN-MY_IDcvv/lib/python3.8/site-packages/pandas/core/frame.py:4162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,County Orig,Year,Layoff/Closure clean
0,06/09/2020,06/07/2020,07/01/2020,Bay Club Redondo Beach,Redondo Beach,Los Angeles County,102.0,Layoff Permanent,Los Angeles County,2020,layoff permanent
1,06/09/2020,06/07/2020,07/01/2020,Bay Club Rolling Hills,Rolling Hills Estates,Los Angeles County,64.0,Layoff Permanent,Los Angeles County,2020,layoff permanent
2,06/09/2020,06/07/2020,07/01/2020,Bay Club Santa Monica,Santa Monica,Los Angeles County,82.0,Layoff Permanent,Los Angeles County,2020,layoff permanent
3,06/19/2020,08/21/2020,07/01/2020,"Weber Metals, Inc",Paramount,Los Angeles County,169.0,Layoff Permanent,Los Angeles County,2020,layoff permanent
4,06/09/2020,06/07/2020,07/01/2020,StoneTree Golf Club,Novato,Marin County,32.0,Layoff Permanent,Marin County,2020,layoff permanent


### Exporting Standardized Data

Now that we have 2020 and 2019 data in an isolated dataframe, we can export it and clean the company names in Open Refine.
The data with clean company names will be re-read into the notebook on the line after the export. 

In [15]:
# Export File to clean company names in open refine.
cali_2020_2019.to_csv('tobecleaned_california_warn.csv')

In [16]:
### Re-import clean company names file.
open_refine_clean = pd.read_csv('cleaned_in_OR/cleaned_cali_warn_raw.csv')
open_refine_clean.drop(columns='Column', inplace=True)
open_refine_clean.head()

,Notice Date,Effective Date,Received Date,Company,Company 2,City,County,Employees,Layoff/Closure,County Orig,Year,Layoff/Closure clean
0,06/09/2020,06/07/2020,07/01/2020,Bay Club Redondo Beach,Bay Club,Redondo Beach,Los Angeles County,102.0,Layoff Permanent,Los Angeles County,2020,layoff permanent
1,06/09/2020,06/07/2020,07/01/2020,Bay Club Rolling Hills,Bay Club,Rolling Hills Estates,Los Angeles County,64.0,Layoff Permanent,Los Angeles County,2020,layoff permanent
2,06/09/2020,06/07/2020,07/01/2020,Bay Club Santa Monica,Bay Club,Santa Monica,Los Angeles County,82.0,Layoff Permanent,Los Angeles County,2020,layoff permanent
3,06/19/2020,08/21/2020,07/01/2020,"Weber Metals, Inc","Weber Metals, Inc.",Paramount,Los Angeles County,169.0,Layoff Permanent,Los Angeles County,2020,layoff permanent
4,06/09/2020,06/07/2020,07/01/2020,StoneTree Golf Club,StoneTree Golf Club,Novato,Marin County,32.0,Layoff Permanent,Marin County,2020,layoff permanent


### Adding Population Data

The data has been grabbed from the census link above. These figures will be used to calculate per capita layoff numbers per county.

In [17]:
cali_population.rename(columns={'California':'Counties','39,512,223':'Population'}, inplace=True)
cali_population['County Names'] = cali_population['Counties'].str[1:]
cali_population['County Names'] = cali_population['County Names'].str.split(',').str[0]
cali_population.head(4)

,Counties,Population,County Names
0,".Alameda County, California",1671329,Alameda County
1,".Alpine County, California",1129,Alpine County
2,".Amador County, California",39752,Amador County
3,".Butte County, California",219186,Butte County


In [18]:
cali_merge = pd.merge(open_refine_clean, cali_population, how="left", left_on="County", right_on="County Names")
cali_merge.drop(columns='Counties', inplace=True)
cali_merge['Year'] = cali_merge['Year'].astype(np.int64)
cali_merge['City 2'] = cali_merge['City'].str.lower()
cali_merge.head()

,Notice Date,Effective Date,Received Date,Company,Company 2,City,County,Employees,Layoff/Closure,County Orig,Year,Layoff/Closure clean,Population,County Names,City 2
0,06/09/2020,06/07/2020,07/01/2020,Bay Club Redondo Beach,Bay Club,Redondo Beach,Los Angeles County,102.0,Layoff Permanent,Los Angeles County,2020,layoff permanent,10039107,Los Angeles County,redondo beach
1,06/09/2020,06/07/2020,07/01/2020,Bay Club Rolling Hills,Bay Club,Rolling Hills Estates,Los Angeles County,64.0,Layoff Permanent,Los Angeles County,2020,layoff permanent,10039107,Los Angeles County,rolling hills estates
2,06/09/2020,06/07/2020,07/01/2020,Bay Club Santa Monica,Bay Club,Santa Monica,Los Angeles County,82.0,Layoff Permanent,Los Angeles County,2020,layoff permanent,10039107,Los Angeles County,santa monica
3,06/19/2020,08/21/2020,07/01/2020,"Weber Metals, Inc","Weber Metals, Inc.",Paramount,Los Angeles County,169.0,Layoff Permanent,Los Angeles County,2020,layoff permanent,10039107,Los Angeles County,paramount
4,06/09/2020,06/07/2020,07/01/2020,StoneTree Golf Club,StoneTree Golf Club,Novato,Marin County,32.0,Layoff Permanent,Marin County,2020,layoff permanent,258826,Marin County,novato


### Removing Possible Duplicates

Code block 19 removes duplicate data. This means that numbers derived from this analysis will be an undercount. 

In [19]:
# Removing duplicate data
dupes = cali_merge[cali_merge[['Company 2', 'City 2', 'County', 'Employees', 'Year']].duplicated(keep=False)]
dupes.sort_values(by='Company 2', ascending=True)
dupes.head()

,Notice Date,Effective Date,Received Date,Company,Company 2,City,County,Employees,Layoff/Closure,County Orig,Year,Layoff/Closure clean,Population,County Names,City 2
6,06/23/2020,06/23/2020,07/01/2020,The Freeman Company LLC,"The Freeman Company, LLC",Anaheim,Orange County,29.0,Layoff Permanent,Orange County,2020,layoff permanent,3175692,Orange County,anaheim
41,06/16/2020,03/31/2020,07/02/2020,PLAYERS CASINO,PLAYERS CASINO,Ventura,Ventura County,183.0,Layoff Temporary,Ventura County,2020,layoff temporary,846006,Ventura County,ventura
44,04/16/2020,03/23/2020,07/03/2020,Tri-Mountain,Tri-Mountain,Irwindale,Los Angeles County,59.0,Layoff Temporary,Los Angeles County,2020,layoff temporary,10039107,Los Angeles County,irwindale
54,06/30/2020,05/09/2020,07/03/2020,Golden Valley Health Centers,Golden Valley Health Centers,Ceres,Stanislaus County,2.0,Layoff Permanent,Stanislaus County,2020,layoff permanent,550660,Stanislaus County,ceres
63,07/01/2020,07/01/2020,07/06/2020,ELECTRO RENT,Electro Rent,Canoga Park,Los Angeles County,25.0,Layoff Permanent,Los Angeles County,2020,layoff permanent,10039107,Los Angeles County,canoga park


In [20]:
## Creates DataFrame without the duplicate data, which means we will be undercounting. But all we need to do is state that.

cali_no_dupes = cali_merge[~cali_merge.isin(dupes)].dropna()

In [21]:
len(cali_merge)

6538

In [22]:
len(cali_no_dupes)

6131

### The start of the California WARN Analysis

We are comparing 2019 layoffs from the start of the year until 12/11/19, and 2020 layoffs from the start of the year until 12/11/20. This allows us to have a clear picture of the scope of the impact within the same time frame.

In [23]:
## Number of Laid Off Workers 2019

cali_2019_info = cali_no_dupes[cali_no_dupes['Year'] == 2019]
cali_2019_info['Employees'].sum()

60719.0

In [24]:
## Number of Companies that laid people off 2019 

cali_2019_info['Company 2'].nunique()

466

In [25]:
## Number of Laid Off Workers 2020

cali_2020_info = cali_no_dupes[cali_no_dupes['Year'] == 2020]
cali_2020_info['Employees'].sum()

607193.0

In [26]:
## Number of Companies that laid people off 2020 

cali_2020_info['Company 2'].nunique()

2880

In [27]:
## Number of Laid Off Workers from beginning of 2020 to Dec 11 - which is the most recent data we have.

layoffs_closures_20 = cali_2020_info[cali_2020_info['Notice Date'] < '12/11/2020']
layoffs_closures_20['Employees'].sum()

607193.0

In [28]:
## Number of Laid Off Workers from beginning of 2019 to Dec 11 - which is the time frame for the data from 2020.

layoffs_closures_20 = cali_2019_info[cali_2019_info['Notice Date'] < '12/11/2020']
layoffs_closures_20['Employees'].sum()

59990.0

In [29]:
## Figures for closures/layoffs in 2019

figures_2019 = cali_2019_info.groupby(['Layoff/Closure']).sum()
figures_2019.reset_index()
figures_2019.drop(columns=['Year','Population'])

,Employees
Layoff/Closure,
Closure Permanent,27928.0
Closure Temporary,3233.0
Closure Unknown at this time,1233.0
Closure Unknown at thistime,272.0
Layoff Permanent,24185.0
Layoff Temporary,1913.0
Layoff Unknown at this time,1361.0
Layoff Unknown at thistime,594.0


In [30]:
## Figures for closures/layoffs in 2020

figures_2020 = cali_2020_info.groupby(['Layoff/Closure']).sum()
figures_2020.reset_index()
figures_2020.drop(columns=['Year','Population'])

,Employees
Layoff/Closure,
Closure Permanent,28212.0
Closure Temporary,70641.0
Closure Type Unknown,598.0
Closure Unknown at thistime,2651.0
Layoff Permanent,107219.0
Layoff Temporary,373211.0
Layoff Type Unknown,6116.0
Layoff Unknown at thistime,18545.0


In [31]:
layoffs = ['layoff permanent', 'layoff temporary', 'layoff type unknown']
closures = ['closure permanent', 'closure temporary', 'closure type unknown']

In [32]:
# of all layoff types (perm, temp, unknown) for 2019

layoffs_2019 = cali_2019_info[cali_2019_info['Layoff/Closure clean'].isin(layoffs)]
layoffs_2019['Employees'].sum()

28053.0

In [33]:
# of all closure types (perm, temp, unknown) for 2019

closures_2019 = cali_2019_info[cali_2019_info['Layoff/Closure clean'].isin(closures)]
closures_2019['Employees'].sum()

32666.0

In [34]:
# of all layoff types (perm, temp, unknown) for 2020

layoffs_2020 = cali_2020_info[cali_2020_info['Layoff/Closure clean'].isin(layoffs)]
layoffs_2020['Employees'].sum()

505091.0

In [35]:
# of all closure types (perm, temp, unknown) for 2020

closures_2020 = cali_2020_info[cali_2020_info['Layoff/Closure clean'].isin(closures)]
closures_2020['Employees'].sum()

102102.0

In [36]:
# check that they add up

layoffs_2020['Employees'].sum() + closures_2020['Employees'].sum()

607193.0

In [37]:
layoffs_2019['Employees'].sum() + closures_2019['Employees'].sum()

60719.0

### Calculating Total Layoffs + Closures Per Capita

In [38]:
# Calculations for 2019

employees_sum_2019 = cali_2019_info[['County','Employees']]
layoffs_sum_2019 = employees_sum_2019.groupby('County').agg('sum')
layoffs_sum_2019.reset_index()

,County,Employees
0,Alameda County,3741.0
1,Butte County,1874.0
2,Contra Costa County,1808.0
3,Fresno County,1189.0
4,Glenn County,143.0
5,Imperial County,44.0
6,Kern County,1523.0
7,Los Angeles County,14307.0
8,Mendocino County,80.0
9,Napa County,38.0


In [39]:
cali_info_and_sums_19 = pd.merge(cali_2019_info, layoffs_sum_2019, how="left", left_on="County", right_on="County")
cali_info_and_sums_19.rename(columns={'Employees_y':'Total Emp. Laid Off in County', 'Employees_x':'Employees'}, inplace=True)
cali_info_and_sums_19.head()

,Notice Date,Effective Date,Received Date,Company,Company 2,City,County,Employees,Layoff/Closure,County Orig,Year,Layoff/Closure clean,Population,County Names,City 2,Total Emp. Laid Off in County
0,11/22/2019,01/21/2020,09/30/2020,"WeWork Companies, Inc.","WeWork Companies, Inc.",San Francisco,San Francisco County,326.0,Layoff Permanent,San Francisco County,2019.0,layoff permanent,881549.0,San Francisco County,san francisco,4666.0
1,04/29/2019,07/01/2019,03/03/2020,Kimberly-Clark Corporation,Kimberly-Clark Corporation,Fullerton,Orange County,204.0,Closure Permanent,Orange County,2019.0,closure permanent,3175692.0,Orange County,fullerton,3831.0
2,05/02/2019,07/05/2019,02/25/2020,Alticor Inc.,Alticor Inc.,Buena Park,Orange County,45.0,Layoff Permanent,Orange County,2019.0,layoff permanent,3175692.0,Orange County,buena park,3831.0
3,12/31/2019,03/03/2020,01/02/2020,"Le Boulanger, Inc.","Le Boulanger, Inc.",Sunnyvale,Santa Clara County,143.0,Closure Permanent,Santa Clara County,2019.0,closure permanent,1927852.0,Santa Clara County,sunnyvale,7365.0
4,12/19/2019,02/13/2020,12/30/2019,NortonLifeLock Inc.,NortonLifeLock Inc.,Mountain View,Santa Clara County,19.0,Layoff Permanent,Santa Clara County,2019.0,layoff permanent,1927852.0,Santa Clara County,mountain view,7365.0


In [40]:
cali_info_and_sums_19['Per Capita Laid Off 2019'] = (cali_info_and_sums_19['Total Emp. Laid Off in County']/cali_info_and_sums_19['Population']) * 100000
per_capita_laid_off = cali_info_and_sums_19[['County', 'Per Capita Laid Off 2019']]
per_capita_laid_off.groupby("County").max().sort_values(by='Per Capita Laid Off 2019', ascending=False)

,Per Capita Laid Off 2019
County,
Butte County,854.981614
San Francisco County,529.295592
Glenn County,503.645265
Santa Clara County,382.031401
Nevada County,244.599268
Alameda County,223.833847
San Mateo County,208.068899
Yolo County,200.907029
Placer County,189.792860


In [41]:
# Calculations for 2020

employees_sum_2020 = cali_2020_info[['County','Employees']]
layoffs_sum_2020 = employees_sum_2020.groupby('County').agg('sum')
layoffs_sum_2020.reset_index()

,County,Employees
0,Alameda County,32767.0
1,Amador County,159.0
2,Butte County,1631.0
3,Calaveras County,40.0
4,Colusa County,30.0
5,Contra Costa County,7693.0
6,Del Norte County,6.0
7,El Dorado County,819.0
8,Fresno County,5809.0
9,Glenn County,47.0


In [42]:
cali_info_and_sums_20 = pd.merge(cali_2020_info, layoffs_sum_2020, how="left", left_on="County", right_on="County")
cali_info_and_sums_20.rename(columns={'Employees_y':'Total Emp. Laid Off in County', 'Employees_x':'Employees'}, inplace=True)
cali_info_and_sums_20.head()

,Notice Date,Effective Date,Received Date,Company,Company 2,City,County,Employees,Layoff/Closure,County Orig,Year,Layoff/Closure clean,Population,County Names,City 2,Total Emp. Laid Off in County
0,06/09/2020,06/07/2020,07/01/2020,Bay Club Redondo Beach,Bay Club,Redondo Beach,Los Angeles County,102.0,Layoff Permanent,Los Angeles County,2020.0,layoff permanent,10039107.0,Los Angeles County,redondo beach,187525.0
1,06/09/2020,06/07/2020,07/01/2020,Bay Club Rolling Hills,Bay Club,Rolling Hills Estates,Los Angeles County,64.0,Layoff Permanent,Los Angeles County,2020.0,layoff permanent,10039107.0,Los Angeles County,rolling hills estates,187525.0
2,06/09/2020,06/07/2020,07/01/2020,Bay Club Santa Monica,Bay Club,Santa Monica,Los Angeles County,82.0,Layoff Permanent,Los Angeles County,2020.0,layoff permanent,10039107.0,Los Angeles County,santa monica,187525.0
3,06/19/2020,08/21/2020,07/01/2020,"Weber Metals, Inc","Weber Metals, Inc.",Paramount,Los Angeles County,169.0,Layoff Permanent,Los Angeles County,2020.0,layoff permanent,10039107.0,Los Angeles County,paramount,187525.0
4,06/09/2020,06/07/2020,07/01/2020,StoneTree Golf Club,StoneTree Golf Club,Novato,Marin County,32.0,Layoff Permanent,Marin County,2020.0,layoff permanent,258826.0,Marin County,novato,2385.0


In [43]:
cali_info_and_sums_20['Per Capita Laid Off 2020'] = (cali_info_and_sums_20['Total Emp. Laid Off in County']/cali_info_and_sums_20['Population']) * 100000
per_capita_laid_off_20 = cali_info_and_sums_20[['County', 'Per Capita Laid Off 2020']]
per_capita_laid_off_20.groupby("County").max().sort_values(by='Per Capita Laid Off 2020', ascending=False)

,Per Capita Laid Off 2020
County,
Mariposa County,13148.869383
Mono County,6085.571864
San Francisco County,5991.499055
Inyo County,2987.970508
San Diego County,2316.936912
San Mateo County,2162.481590
Napa County,2056.713904
Monterey County,1981.288344
Alameda County,1960.535598


In [44]:
per_capitas = pd.merge(per_capita_laid_off_20, per_capita_laid_off, how='left', left_on="County", right_on="County")
per_capitas.groupby("County").max().sort_values(by='Per Capita Laid Off 2020', ascending=False)

,Per Capita Laid Off 2020,Per Capita Laid Off 2019
County,,
Mariposa County,13148.869383,NaN
Mono County,6085.571864,NaN
San Francisco County,5991.499055,529.295592
Inyo County,2987.970508,NaN
San Diego County,2316.936912,114.847843
San Mateo County,2162.481590,208.068899
Napa County,2056.713904,27.587409
Monterey County,1981.288344,NaN
Alameda County,1960.535598,223.833847


In [45]:
layoff_closure_list_perm = ['layoff permanent', 'closure permanent']
layoff_closure_list_temp = ['layoff temporary', 'closure temporary']
counties_list = ['Butte County', 'Santa Cruz County', 'Yolo County', 'Orange County', 'Los Angeles County', 'Humboldt County', 'Fresno County', 'Mariposa County']

In [46]:
high_rent_burden_20_prm = cali_2020_info[cali_2020_info['County'].isin(counties_list)]
high_rent_burden_20_prm = high_rent_burden_20_prm[high_rent_burden_20_prm['Layoff/Closure clean'].isin(layoff_closure_list_perm)]
high_rent_burden_20_prm = high_rent_burden_20_prm[['County','Layoff/Closure','Employees']]
high_rent_burden_20_prm = high_rent_burden_20_prm.groupby(['County','Layoff/Closure']).sum()
high_rent_burden_20_prm.reset_index('Layoff/Closure')

,Layoff/Closure,Employees
County,,
Butte County,Closure Permanent,14.0
Butte County,Layoff Permanent,570.0
Fresno County,Closure Permanent,980.0
Fresno County,Layoff Permanent,471.0
Los Angeles County,Closure Permanent,10696.0
Los Angeles County,Layoff Permanent,29542.0
Mariposa County,Layoff Permanent,45.0
Orange County,Closure Permanent,1811.0
Orange County,Layoff Permanent,11073.0


In [47]:
high_rent_burden_19_prm = cali_2019_info[cali_2019_info['County'].isin(counties_list)]
high_rent_burden_19_prm = high_rent_burden_19_prm[high_rent_burden_19_prm['Layoff/Closure clean'].isin(layoff_closure_list_perm)]
high_rent_burden_19_prm = high_rent_burden_19_prm[['County','Layoff/Closure','Employees']]
high_rent_burden_19_prm = high_rent_burden_19_prm.groupby(['County','Layoff/Closure']).sum()
high_rent_burden_19_prm.reset_index('Layoff/Closure')

,Layoff/Closure,Employees
County,,
Butte County,Closure Permanent,501.0
Butte County,Layoff Permanent,42.0
Fresno County,Closure Permanent,1160.0
Fresno County,Layoff Permanent,10.0
Los Angeles County,Closure Permanent,7382.0
Los Angeles County,Layoff Permanent,5795.0
Orange County,Closure Permanent,2078.0
Orange County,Layoff Permanent,1501.0
Santa Cruz County,Closure Permanent,137.0


In [48]:
perm_job_loss = pd.merge(high_rent_burden_20_prm, high_rent_burden_19_prm, how='left', left_on=["County", "Layoff/Closure"], right_on=["County", "Layoff/Closure"])
perm_job_loss.rename(columns={'Employees_x':'Employees Laid Off 2020', 'Employees_y':'Employees Laid Off 2019'}, inplace=True)

In [49]:
high_rent_burden_20_tmp = cali_2020_info[cali_2020_info['County'].isin(counties_list)]
high_rent_burden_20_tmp = high_rent_burden_20_tmp[high_rent_burden_20_tmp['Layoff/Closure clean'].isin(layoff_closure_list_temp)]
high_rent_burden_20_tmp = high_rent_burden_20_tmp[['County','Layoff/Closure','Employees']]
high_rent_burden_20_tmp = high_rent_burden_20_tmp.groupby(['County','Layoff/Closure']).sum()
high_rent_burden_20_tmp.reset_index('Layoff/Closure')

,Layoff/Closure,Employees
County,,
Butte County,Layoff Temporary,1002.0
Fresno County,Closure Temporary,882.0
Fresno County,Layoff Temporary,3476.0
Humboldt County,Layoff Temporary,227.0
Los Angeles County,Closure Temporary,19201.0
Los Angeles County,Layoff Temporary,118829.0
Mariposa County,Closure Temporary,343.0
Mariposa County,Layoff Temporary,1874.0
Orange County,Closure Temporary,6738.0


In [50]:
high_rent_burden_19_tmp = cali_2019_info[cali_2019_info['County'].isin(counties_list)]
high_rent_burden_19_tmp = high_rent_burden_19_tmp[high_rent_burden_19_tmp['Layoff/Closure clean'].isin(layoff_closure_list_temp)]
high_rent_burden_19_tmp = high_rent_burden_19_tmp[['County','Layoff/Closure','Employees']]
high_rent_burden_19_tmp = high_rent_burden_19_tmp.groupby(['County','Layoff/Closure']).sum()
high_rent_burden_19_tmp.reset_index('Layoff/Closure')

,Layoff/Closure,Employees
County,,
Butte County,Closure Temporary,1331.0
Los Angeles County,Closure Temporary,326.0
Los Angeles County,Layoff Temporary,338.0
Santa Cruz County,Layoff Temporary,93.0


In [51]:
temp_job_loss = pd.merge(high_rent_burden_20_tmp, high_rent_burden_19_tmp, how='left', left_on=["County", "Layoff/Closure"], right_on=["County", "Layoff/Closure"])
temp_job_loss.rename(columns={'Employees_x':'Employees Laid Off 2020', 'Employees_y':'Employees Laid Off 2019'}, inplace=True)

### Temporary and Permanent Job Loss with 2020 and 2019 Comparisons

In [52]:
temp_job_loss

Employees Laid Off 2020  \
County             Layoff/Closure                               
Butte County       Layoff Temporary                    1002.0   
Fresno County      Closure Temporary                    882.0   
                   Layoff Temporary                    3476.0   
Humboldt County    Layoff Temporary                     227.0   
Los Angeles County Closure Temporary                  19201.0   
                   Layoff Temporary                  118829.0   
Mariposa County    Closure Temporary                    343.0   
                   Layoff Temporary                    1874.0   
Orange County      Closure Temporary                   6738.0   
                   Layoff Temporary                   38186.0   
Santa Cruz County  Closure Temporary                    253.0   
                   Layoff Temporary                    2702.0   
Yolo County        Closure Temporary                     24.0   
                   Layoff Temporary                     599.0   

                                      Employees Laid Off 2019  
County             Layoff/Closure                              
Butte County       Layoff Temporary                       NaN  
Fresno County      Closure Temporary                      NaN  
                   Layoff Temporary                       NaN  
Humboldt County    Layoff Temporary                       NaN  
Los Angeles County Closure Temporary                    326.0  
                   Layoff Temporary                     338.0  
Mariposa County    Closure Temporary                      NaN  
                   Layoff Temporary                       NaN  
Orange County      Closure Temporary                      NaN  
                   Layoff Temporary                       NaN  
Santa Cruz County  Closure Temporary                      NaN  
                   Layoff Temporary                      93.0  
Yolo County        Closure Temporary                      NaN  
                   Layoff Temporary                       NaN

In [53]:
perm_job_loss

Employees Laid Off 2020  \
County             Layoff/Closure                               
Butte County       Closure Permanent                     14.0   
                   Layoff Permanent                     570.0   
Fresno County      Closure Permanent                    980.0   
                   Layoff Permanent                     471.0   
Los Angeles County Closure Permanent                  10696.0   
                   Layoff Permanent                   29542.0   
Mariposa County    Layoff Permanent                      45.0   
Orange County      Closure Permanent                   1811.0   
                   Layoff Permanent                   11073.0   
Santa Cruz County  Closure Permanent                    447.0   
                   Layoff Permanent                     988.0   
Yolo County        Closure Permanent                     84.0   
                   Layoff Permanent                     457.0   

                                      Employees Laid Off 2019  
County             Layoff/Closure                              
Butte County       Closure Permanent                    501.0  
                   Layoff Permanent                      42.0  
Fresno County      Closure Permanent                   1160.0  
                   Layoff Permanent                      10.0  
Los Angeles County Closure Permanent                   7382.0  
                   Layoff Permanent                    5795.0  
Mariposa County    Layoff Permanent                       NaN  
Orange County      Closure Permanent                   2078.0  
                   Layoff Permanent                    1501.0  
Santa Cruz County  Closure Permanent                    137.0  
                   Layoff Permanent                       NaN  
Yolo County        Closure Permanent                    443.0  
                   Layoff Permanent                       NaN

In [54]:
high_rent_burden_19_prm = cali_2019_info[cali_2019_info['County'].isin(counties_list)]
high_rent_burden_19_prm = high_rent_burden_19_prm[high_rent_burden_19_prm['Layoff/Closure clean'].isin(layoff_closure_list_perm)]
perm_job_loss_total = high_rent_burden_19_prm[['County','Layoff/Closure','Employees']]
perm_job_loss_total = perm_job_loss_total.groupby(['County']).sum()
perm_job_loss_total

,Employees
County,
Butte County,543.0
Fresno County,1170.0
Los Angeles County,13177.0
Orange County,3579.0
Santa Cruz County,137.0
Yolo County,443.0


In [55]:
high_rent_burden_20_prm = cali_2020_info[cali_2020_info['County'].isin(counties_list)]
high_rent_burden_20_prm = high_rent_burden_20_prm[high_rent_burden_20_prm['Layoff/Closure clean'].isin(layoff_closure_list_perm)]
perm_job_loss_total_20 = high_rent_burden_20_prm[['County','Layoff/Closure','Employees']]
perm_job_loss_total_20 = perm_job_loss_total_20.groupby(['County']).sum()
perm_job_loss_total_20

,Employees
County,
Butte County,584.0
Fresno County,1451.0
Los Angeles County,40238.0
Mariposa County,45.0
Orange County,12884.0
Santa Cruz County,1435.0
Yolo County,541.0


In [56]:
sums_2020_2019 = perm_job_loss_total_20.merge(perm_job_loss_total, how="left", left_on="County", right_on="County")
sums_2020_2019.rename(columns={'Employees_x':'Permanent L/C Total 2020','Employees_y':'Permanent L/C Total 2019'}, inplace=True)
sums_2020_2019

,Permanent L/C Total 2020,Permanent L/C Total 2019
County,,
Butte County,584.0,543.0
Fresno County,1451.0,1170.0
Los Angeles County,40238.0,13177.0
Mariposa County,45.0,NaN
Orange County,12884.0,3579.0
Santa Cruz County,1435.0,137.0
Yolo County,541.0,443.0


In [57]:
#Fact Check for # of Layoffs in Mariposa County

Mariposa = cali_2020_info[cali_2020_info['County'] == 'Mariposa County']
Mariposa.groupby('County').sum() # Laid Off Employees totals 2,262

,Employees,Year,Population
County,,,
Mariposa County,2262.0,12120.0,103218.0


In [58]:
Mariposa.groupby('County').max() # Population totals 17,203

,Notice Date,Effective Date,Received Date,Company,Company 2,City,Employees,Layoff/Closure,County Orig,Year,Layoff/Closure clean,Population,County Names,City 2
County,,,,,,,,,,,,,,
Mariposa County,07/30/2020,08/15/2020,08/03/2020,NatureBridge at Yosemite,NatureBridge,Yosemite National Park,1829.0,Layoff Temporary,Mariposa County,2020.0,layoff temporary,17203.0,Mariposa County,yosemite national park


In [59]:
Mariposa = cali_2020_info[cali_2020_info['County'] == 'Mariposa County']
Mariposa

,Notice Date,Effective Date,Received Date,Company,Company 2,City,County,Employees,Layoff/Closure,County Orig,Year,Layoff/Closure clean,Population,County Names,City 2
328,07/30/2020,08/15/2020,08/03/2020,NatureBridge at Yosemite,NatureBridge,Yosemite National Park,Mariposa County,45.0,Layoff Permanent,Mariposa County,2020.0,layoff permanent,17203.0,Mariposa County,yosemite national park
1913,04/29/2020,03/22/2020,06/06/2020,El Portal Canyon Restaurant dba Miner'sRoadho...,El Portal Canyon Restaurant dba Miner'sRoadho...,Mariposa,Mariposa County,15.0,Layoff Temporary,Mariposa County,2020.0,layoff temporary,17203.0,Mariposa County,mariposa
1938,05/11/2020,03/26/2020,06/05/2020,El Portal Dining Inc dba Cedar House CanyonBar...,El Portal Dining Inc,El Portal,Mariposa County,9.0,Layoff Temporary,Mariposa County,2020.0,layoff temporary,17203.0,Mariposa County,el portal
1973,05/11/2020,03/26/2020,06/05/2020,El Portal Dining Inc dba Parkside Pizza,El Portal Dining Inc,El Portal,Mariposa County,21.0,Layoff Temporary,Mariposa County,2020.0,layoff temporary,17203.0,Mariposa County,el portal
2021,04/15/2020,03/19/2020,06/04/2020,Delaware North Companies Parks & Resorts atTen...,Delaware North Companies Parks & Resorts atTen...,Fish Camp,Mariposa County,343.0,Closure Temporary,Mariposa County,2020.0,closure temporary,17203.0,Mariposa County,fish camp
3967,04/07/2020,03/13/2020,04/20/2020,Aramark - Yosemite,"Aramark Services, Inc.",Yosemite National Park,Mariposa County,1829.0,Layoff Temporary,Mariposa County,2020.0,layoff temporary,17203.0,Mariposa County,yosemite national park


In [60]:
# Fact Check that West Sacramento is most affected by Layoffs in Yolo County

Yolo = cali_2020_info[cali_2020_info['County'] == 'Yolo County']
Yolo = Yolo[Yolo['Year'] == 2020]
Yolo.groupby('City').sum()

,Employees,Year,Population
City,,,
Davis,290.0,28280.0,3087000.0
Madison,8.0,2020.0,220500.0
West Sacramento,704.0,26260.0,2866500.0
Winters,111.0,4040.0,441000.0
Woodland,197.0,12120.0,1323000.0


In [61]:
## Fact-Checking how many Layoffs Driven Performance Brands had in total for 2020.

Yolo = cali_2020_info[cali_2020_info['County'] == 'Yolo County']
sum_it = Yolo[Yolo['Company 2'] == 'Driven Performance Brands, Inc.']
sum_it.groupby('County').sum()

,Employees,Year,Population
County,,,
Yolo County,240.0,6060.0,661500.0


In [62]:
## Fact Check Santa Cruz County's permanent layoff figures

santa_cruz_20 = cali_2020_info[cali_2020_info['County'] == 'Santa Cruz County'] 
santa_cruz_20.groupby('County').max()

,Notice Date,Effective Date,Received Date,Company,Company 2,City,Employees,Layoff/Closure,County Orig,Year,Layoff/Closure clean,Population,County Names,City 2
County,,,,,,,,,,,,,,
Santa Cruz County,11/18/2020,11/23/2020,12/01/2020,Watsonville Community Hospital,Watsonville Community Hospital,Watsonville,1097.0,Layoff Unknown at thistime,Santa Cruz County,2020.0,layoff type unknown,273213.0,Santa Cruz County,watsonville


In [63]:
santa_cruz_20_perm = santa_cruz_20[santa_cruz_20['Layoff/Closure clean'] == 'layoff permanent']
santa_cruz_20_perm.groupby('County').max()

#Just extract employee field

,Notice Date,Effective Date,Received Date,Company,Company 2,City,Employees,Layoff/Closure,County Orig,Year,Layoff/Closure clean,Population,County Names,City 2
County,,,,,,,,,,,,,,
Santa Cruz County,10/28/2020,10/26/2020,11/16/2020,Watsonville Community Hospital,Watsonville Community Hospital,Watsonville,265.0,Layoff Permanent,Santa Cruz County,2020.0,layoff permanent,273213.0,Santa Cruz County,watsonville


In [64]:
santa_cruz_20_perm[santa_cruz_20_perm['Employees'] == 265.0]

,Notice Date,Effective Date,Received Date,Company,Company 2,City,County,Employees,Layoff/Closure,County Orig,Year,Layoff/Closure clean,Population,County Names,City 2
1151,07/31/2020,09/01/2020,10/12/2020,"Mount Hermon Association, Inc.","Mount Hermon Association, Inc",Felton,Santa Cruz County,265.0,Layoff Permanent,Santa Cruz County,2020.0,layoff permanent,273213.0,Santa Cruz County,felton


In [65]:
## No Layoffs from Mount Hermon Association, so it seems like the layoffs for the company in 2020 are not a seasonal thing

santa_cruz_19 = cali_2019_info[cali_2019_info['County'] == 'Santa Cruz County']
santa_cruz_19

,Notice Date,Effective Date,Received Date,Company,Company 2,City,County,Employees,Layoff/Closure,County Orig,Year,Layoff/Closure clean,Population,County Names,City 2
5938,10/09/2019,12/09/2019,10/11/2019,Chaminade Resort & Spa,CHAMINADE RESORT AND SPA,Santa Cruz,Santa Cruz County,93.0,Layoff Temporary,Santa Cruz County,2019.0,layoff temporary,273213.0,Santa Cruz County,santa cruz
5998,08/30/2019,10/31/2019,09/06/2019,"Coastline Support, Inc.","Coastline Support, Inc.",Santa Cruz,Santa Cruz County,55.0,Closure Permanent,Santa Cruz County,2019.0,closure permanent,273213.0,Santa Cruz County,santa cruz
6319,04/12/2019,06/12/2019,04/12/2019,"Spawn Mate, Inc. dba Mushroom Farms","Spawn Mate, Inc. dba Mushroom Farms",Watsonville,Santa Cruz County,82.0,Closure Permanent,Santa Cruz,2019.0,closure permanent,273213.0,Santa Cruz County,watsonville


In [66]:
## Looking into Butte County, which had the highest rent burden

butte = cali_2020_info[cali_2020_info['County'] == 'Butte County']
butte

,Notice Date,Effective Date,Received Date,Company,Company 2,City,County,Employees,Layoff/Closure,County Orig,Year,Layoff/Closure clean,Population,County Names,City 2
627,04/23/2020,04/24/2020,09/01/2020,"Lulu's Fashion Lounge, LLC","Lulu's Fashion Lounge, LLC",Chico,Butte County,78.0,Layoff Permanent,Butte County,2020.0,layoff permanent,219186.0,Butte County,chico
1024,09/25/2020,08/19/2020,09/29/2020,"CSU, Chico","CSU, Chico",Chico,Butte County,446.0,Layoff Permanent,Butte County,2020.0,layoff permanent,219186.0,Butte County,chico
1429,11/16/2020,01/11/2021,11/23/2020,Adventist Health Home Care,Adventist Health Bakersfield,Chico,Butte County,14.0,Closure Permanent,Butte County,2020.0,closure permanent,219186.0,Butte County,chico
2202,04/21/2020,04/10/2020,05/27/2020,Associated Students of California StateUnivers...,Associated Students California State Universit...,Chico,Butte County,660.0,Layoff Temporary,Butte County,2020.0,layoff temporary,219186.0,Butte County,chico
2741,03/23/2020,03/23/2020,05/14/2020,"Lulu's Fashion Lounge, LLC","Lulu's Fashion Lounge, LLC",Chico,Butte County,213.0,Layoff Temporary,Butte County,2020.0,layoff temporary,219186.0,Butte County,chico
3305,04/08/2020,03/17/2020,04/29/2020,Restec Contractors Inc. at CSU Chico,Restec Contractors Inc. at Bayview,Chico,Butte County,8.0,Layoff Temporary,Butte County,2020.0,layoff temporary,219186.0,Butte County,chico
3410,04/17/2020,04/12/2020,04/28/2020,Dick's Sporting Goods Inc. at Chico Store#1089,"Dick's Sporting Goods, Inc.",Chico,Butte County,28.0,Layoff Temporary,Butte County,2020.0,layoff temporary,219186.0,Butte County,chico
3975,04/02/2020,03/24/2020,04/20/2020,Pancakes 1702 INC,"Pancakes, Inc.",Chico,Butte County,45.0,Layoff Unknown at thistime,Butte County,2020.0,layoff type unknown,219186.0,Butte County,chico
4378,03/26/2020,03/26/2020,04/15/2020,Cinemark USA Inc. dba Cinemark Chico,Cinemark USA Inc.,Chico,Butte County,46.0,Layoff Permanent,Butte County,2020.0,layoff permanent,219186.0,Butte County,chico
4604,03/30/2020,03/16/2020,04/13/2020,"Hupp Draft Services, Inc.","Hupp Draft Services, Inc.",Chico,Butte County,93.0,Layoff Temporary,Butte County,2020.0,layoff temporary,219186.0,Butte County,chico


In [68]:
butte_19 = cali_2019_info[cali_2019_info['County'] == 'Butte County']
butte_19 = butte_19[butte_19['Year'] == 2019]
butte_19

,Notice Date,Effective Date,Received Date,Company,Company 2,City,County,Employees,Layoff/Closure,County Orig,Year,Layoff/Closure clean,Population,County Names,City 2
5820,12/18/2019,02/01/2020,12/19/2019,Adventist Health Feather River Palliative Care,Adventist Health Bakersfield,Chico,Butte County,8.0,Closure Permanent,Butte County,2019.0,closure permanent,219186.0,Butte County,chico
5910,10/28/2019,12/27/2019,10/29/2019,Lifetouch National School Studios Inc.,Lifetouch National School Studios Inc.,Chico,Butte County,320.0,Closure Permanent,Butte County,2019.0,closure permanent,219186.0,Butte County,chico
6057,07/22/2019,09/20/2019,07/29/2019,"Milestone Technologies, Inc.","Milestone Technologies, Inc.",Chico,Butte County,148.0,Closure Permanent,Butte County,2019.0,closure permanent,219186.0,Butte County,chico
6100,01/18/2019,03/17/2019,01/23/2019,Cerner Corporation,Cerner Corporation,Paradise,Butte County,42.0,Layoff Permanent,Butte County,2019.0,layoff permanent,219186.0,Butte County,paradise
6107,01/08/2019,02/05/2019,01/25/2019,Adventist Health Feather River,Adventist Health Bakersfield,Paradise,Butte County,1331.0,Closure Temporary,Butte County,2019.0,closure temporary,219186.0,Butte County,paradise
6271,03/22/2019,05/24/2019,03/22/2019,Lifetouch National School Studios Inc.,Lifetouch National School Studios Inc.,Chico,Butte County,25.0,Closure Permanent,Butte County,2019.0,closure permanent,219186.0,Butte County,chico
